In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

from sklearn.metrics import mean_squared_error

C:\Users\Tejas\AppData\Local\Temp\ipykernel_8344\1516330197.py:14: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [4]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [5]:
import tensorflow as tf
tf.random.set_seed(7)

In [6]:
dataframe = pd.read_csv('clean_data.csv', usecols=[5], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [8]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [9]:
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [10]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [11]:

def build_model(hp):
    model = Sequential()
    model.add(LSTM(hp.Int('input_unit',min_value=32,max_value=512,step=32), return_sequences=True, input_shape=(1, look_back)))
    for i in range(hp.Int('n_layers', 1, 10)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
    model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=512,step=32)))
    model.add(Dense(1, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential'],default='softmax')))
    
    model.compile(loss='mean_squared_error', optimizer=hp.Choice('optimizers',values=['sgd', 'rmsprop', 'adam', 'adadelta', 'adagrad', 'adamax', 'nadam', 'frtl'],default='sgd'))
    return model
    #10 LSTM layers and one dense layers

In [13]:
from kerastuner.tuners import RandomSearch

tuner= RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=20,
        overwrite=True,
        executions_per_trial=2,
        directory='lstm_27102022'
    )

In [17]:
tboard = tf.keras.callbacks.TensorBoard(log_dir='logs', write_graph=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(
        x=trainX,
        y=trainY,
        epochs=100,
        batch_size=128,
        validation_data=(testX,testY),
        callbacks=[tboard, stop_early]
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
384               |?                 |input_unit
4                 |?                 |n_layers
64                |?                 |lstm_0_units
384               |?                 |layer_2_neurons
sigmoid           |?                 |dense_activation
adagrad           |?                 |optimizers
32                |?                 |lstm_1_units
32                |?                 |lstm_2_units
32                |?                 |lstm_3_units

Epoch 1/100
9/9 [==============================] - 16s 290ms/step - loss: 0.1019 - val_loss: 0.1208
Epoch 2/100
9/9 [==============================] - 0s 51ms/step - loss: 0.1014 - val_loss: 0.1204
Epoch 3/100
9/9 [==============================] - 1s 64ms/step - loss: 0.1010 - val_loss: 0.1200
Epoch 4/100
9/9 [==============================] - 1s 56ms/step - loss: 0.1007 - val_loss: 0.1197
Epoch 5/100
9/9 [==============================] - 1s 57ms/step - l

KeyboardInterrupt: 

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

trainPredict = best_model.predict(trainX)
testPredict = best_model.predict(testX)

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
from tensorflow.keras.models import save_model
save_model(best_model,'AQI_LSTM_new.h5')

In [ ]:
best_model.summary()
